In [35]:
from __future__ import division

In [36]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib

In [37]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [38]:
#store = pd.HDFStore('access_log.h5')

In [39]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [40]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [41]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [42]:
#squid = sq.read_squid_log('access.log')

In [43]:
#squid.tail()

In [44]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [45]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [46]:
#sum(squid.bytes)/Gb

In [89]:
def juniper_log_cleansing(x):
    if str(x).find('=') > 0:
        return str(x)[str(x).find('=')+1:]
    elif str(x).find(':') > 0:
        return str(x)[:str(x).find(':')]    
    else:
        return x

def read_syslog_juniper(filename):
    
    #if (filename)[-3:] == '.gz':
    #    f = hp.open_gz(filename)
    #elif (filename)[-6:] == 'tar.gz':
    #    f = hp.open_tar_gz(filename)
    #else:
    #    f = hp.open_txt(filename)
    
    chunksize = 20000
    total_chunksize = 0
    
    #df =  pd.read_csv('311_100M.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    df = pd.DataFrame()
    
    for c in pd.read_table(filename, sep=' {1,}|"', chunksize=chunksize, iterator=True, engine='python', index_col=False, header=None, parse_dates=[[0, 1, 2], [9,10]]):
            df = pd.concat([df, c])
            total_chunksize += chunksize
            print(str(total_chunksize) + " processed.")
    
    df = df[df[7].str.contains('information') == False]
    
    # restructure the dataframe
    i = [0]
    i.extend(range(3, 4))
    i.extend([1])
    i.extend(range(9, 13))
    i.extend([14])
    i.extend([16])
    i.extend(range(17, 25))
    df = df.iloc[:, i]
    
    df.columns = ['time', 'device_name', 'traffic_start', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port', 'remarks']
    df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']] = df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']].applymap(juniper_log_cleansing)
    
    return df

In [93]:
infile = "C:\Temp\hk-ssg140.log-20150918"

In [94]:
juniper = read_syslog_juniper(infile)

20000 processed.
40000 processed.
60000 processed.
80000 processed.
100000 processed.
120000 processed.
140000 processed.
160000 processed.
180000 processed.
200000 processed.
220000 processed.
240000 processed.
260000 processed.
280000 processed.
300000 processed.
320000 processed.
340000 processed.
360000 processed.
380000 processed.
400000 processed.
420000 processed.
440000 processed.
460000 processed.
480000 processed.
500000 processed.
520000 processed.
540000 processed.
560000 processed.
580000 processed.
600000 processed.
620000 processed.
640000 processed.
660000 processed.
680000 processed.
700000 processed.
720000 processed.
740000 processed.
760000 processed.
780000 processed.
800000 processed.
820000 processed.
840000 processed.
860000 processed.
880000 processed.
900000 processed.
920000 processed.
940000 processed.
960000 processed.
980000 processed.
1000000 processed.
1020000 processed.
1040000 processed.
1060000 processed.
1080000 processed.
1100000 processed.
1120000 

In [97]:
juniper.head()

,time,device_name,traffic_start,duration,policy_no,service,protocol,src_zone,dst_zone,action,sent_size,received_size,src_address,dst_address,src_port,dst_port,remarks
0,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,216.69.185.8,57524,53,src-xlated
1,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,29,dns,17,HK-LAN,Untrust,Permit,0,0,172.20.78.20,192.33.14.30,57436,53,src-xlated
3,2015-09-17 04:02:02,hk-SSG140,2015-09-17 04:13:57,0,37,snmp,17,HK-DMZ,HK-LAN,Deny,0,50,172.21.200.230,172.20.78.123,50835,161,session_id=0
4,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,38,https,6,HK-LAN,HK-DMZ,Permit,1271,2001,172.20.78.77,172.20.113.3,58776,443,src-xlated
5,2015-09-17 04:02:03,hk-SSG140,2015-09-17 04:13:56,3,470,tcp/port:47337,6,HK-LAN,HK-DMZ,Permit,95,58,172.20.78.115,172.24.66.31,443,47337,src-xlated
